In [1]:
import pandas as pd
import numpy as np
import os
import random
import hashlib
from pathos.multiprocessing import ProcessingPool as Pool
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
import json

data_path = '../../analysis/pre'

In [ ]:
# Get all data of alternative sites

site_info_alt = pd.read_csv(data_path + '/data-fetching/alternative-sites/site_info_20230101.csv')
weather_data_alt = pd.read_csv(data_path + '/data-fetching/alternative-sites/df_weather-20230101-20240501.csv')

weather_data_alt.rename(columns={'ts': 'time'}, inplace=True)
weather_data_alt['time'] = pd.to_datetime(weather_data_alt['time'])

power_data_alt = pd.DataFrame()

temp_site_info = []

for index, row in site_info_alt.iterrows():
    site = row
    cluster = 0 # cluster 0 is chosen (= no clustering)
    site_id = site['site_id']
    
    new_entry = {
        'site_id': site_id, 
        'clusters': {
            'location': -1
        },
        'lat': 0, # site['lat'],
        'lng': 0, # site['lng'],
        'zip': site['zip'],
        'country': site['country'],
        'kwp': site['kwp'],
    }
    temp_site_info.append(new_entry)
    site_power_data = pd.read_csv(data_path + f'/data-fetching/alternative-sites/df_{site_id}-20230101-20240501.csv')
    power_data_alt = pd.concat([power_data_alt, site_power_data])

site_info_alt = pd.DataFrame(temp_site_info)
power_data_alt['time'] = pd.to_datetime(power_data_alt['time'])

print('Alternative sites data loaded')
print(f'Power data length: {len(power_data_alt)}')


In [ ]:
# Get all data of main sites

site_info = pd.read_csv(data_path + '/data-fetching/site_info_20230101.csv')
weather_data = pd.read_csv(data_path + '/data-fetching/df_weather-20230101-20240101.csv')
clusters = pd.read_csv(data_path + '/site-clustering//closest_sites.csv')

weather_data.rename(columns={'ts': 'time'}, inplace=True)
weather_data['time'] = pd.to_datetime(weather_data['time'])

power_data = pd.DataFrame()

temp_site_info = []

for index, row in site_info.iterrows():
    site = row
    cluster = clusters[clusters['site_id'] == site['site_id']].iloc[0]
    site_id = site['site_id']
    new_entry = {
        'site_id': site_id, 
        'clusters': {'location': cluster['cluster']},
        'lat': 0, # site['lat'],
        'lng': 0, # site['lng'],
        'zip': site['zip'],
        'country': site['country'],
        'kwp': site['kwp'],
    }
    temp_site_info.append(new_entry)
    site_power_data = pd.read_csv(data_path + f'/data-fetching/df_{site_id}-20230101-20240101.csv')
    power_data = pd.concat([power_data, site_power_data])

site_info = pd.DataFrame(temp_site_info)
power_data['time'] = pd.to_datetime(power_data['time'])

print('Main sites data loaded')
print(f'Power data length: {len(power_data)}')

In [4]:
# concat alternative and main sites
site_info = pd.concat([site_info, site_info_alt], ignore_index=True)
power_data = pd.concat([power_data, power_data_alt], ignore_index=True)
weather_data = pd.concat([weather_data, weather_data_alt], ignore_index=True)

In [ ]:

print('All data loaded')
print(f'Site info length: {len(site_info)}')
print(f'Power data length: {len(power_data)}')
print(f'Weather data length: {len(weather_data)}')

print(site_info)    


In [6]:
def accumulated_day_energy(power_values):
    # check if any none values
    if any([x is None for x in power_values]) or len(power_values) != 96:
        return [None] * 96
    
    power_values = np.array(power_values)

    # Calculate the energy for each interval (15 minutes => 0.25 hours)
    energy_per_interval = power_values * 0.25

    # Calculate the accumulated energy
    accumulated_energy = np.cumsum(energy_per_interval)

    return accumulated_energy

def whole_day_energy(power_values):
    # check if any none values
    if any([x is None for x in power_values]) or len(power_values) != 96:
        return [None] * 96
    
    power_values = np.array(power_values)

    # Calculate the energy for each interval (15 minutes => 0.25 hours)
    energy_per_interval = power_values * 0.25

    # Calculate the accumulated energy
    sm = np.sum(energy_per_interval)
    whole_day_energy = [sm] * 96

    return whole_day_energy

In [8]:
os.makedirs(f'../test_data', exist_ok=True)

df_all = pd.DataFrame()


def process_site(site_id):
    print("Site ID:", site_id)
    site = site_info[site_info['site_id'] == site_id].iloc[0]
    power_data_site = power_data[(power_data['site_id'] == site_id)]
    weather_data_site = weather_data[(weather_data['zip'] == site['zip'])]
    
    power_data_site['time'] = pd.to_datetime(power_data_site['time'])
    weather_data_site['time'] = pd.to_datetime(weather_data_site['time'])

    power_data_site = power_data_site.sort_values('time')


    if len(weather_data_site) != 0:
        site['weather_data'] = True
        
        weather_data_site = weather_data_site.sort_values('time')

        duplicated_weather = []

        for index, row in weather_data_site.iterrows():
            ts = row['time']
            duplicated_weather.append(row)
            for _ in range(3):
                ts += pd.Timedelta(minutes=15)
                new_row = row.copy()
                new_row['time'] = ts
                # Append the duplicated row to the list
                duplicated_weather.append(new_row)
        
        generated_weather = pd.DataFrame(duplicated_weather)
        generated_weather['site_id'] = site_id
        power_data_site['site_id'] = site_id

        df = pd.merge(generated_weather, power_data_site, on=['site_id', 'time'])

        df = df[['time', 'site_id', 'value_key', 'avg', 'solar_rad', 'temp', 'app_temp', 'uv', 'precip', 'rh', 'ghi', 'snow_depth', 'clouds']]


        weather_columns = ['solar_rad', 'temp', 'app_temp', 'uv', 'precip', 'rh', 'ghi', 'snow_depth', 'clouds']
        for row in df.iterrows():
            # get row 1h and 2h before
            row_1h = df[(df['time'] == row[1]['time'] - pd.Timedelta(hours=1)) & (df['value_key'] == row[1]['value_key'])]
            row_2h = df[(df['time'] == row[1]['time'] - pd.Timedelta(hours=2)) & (df['value_key'] == row[1]['value_key'])]
            
            # set values if data is available
            if len(row_1h) != 0:
                for column in weather_columns:
                    df.loc[row[0], f'{column}_1h'] = row_1h[column].values[0]
            else:
                for column in weather_columns:
                    df.loc[row[0], f'{column}_1h'] = np.nan

            if len(row_2h) != 0:
                for column in weather_columns:
                    df.loc[row[0], f'{column}_2h'] = row_2h[column].values[0]
            else:
                for column in weather_columns:
                    df.loc[row[0], f'{column}_2h'] = np.nan
            
    else:
        df = power_data_site[['time', 'site_id', 'value_key', 'avg']]
        site['weather_data'] = False
    
    
    df.sort_values('time', inplace=True)

    for row in df.iterrows():
        row_24h = df[(df['time'] == row[1]['time'] - pd.Timedelta(hours=24)) & (df['value_key'] == row[1]['value_key'])]

        if len(row_24h) != 0:
            df.loc[row[0], 'avg_24h'] = row_24h['avg'].values[0]
        else:
            df.loc[row[0], 'avg_24h'] = np.nan

    days = df['time'].dt.date.unique()
    for day in days:
        for value_key in df['value_key'].unique():
            day_before = day - pd.Timedelta(days=1)
            power_values = df[(df['time'].dt.date == day_before) & (df['value_key'] == value_key)]['avg'].values
            num_rows = len(df[(df['time'].dt.date == day) & (df['value_key'] == value_key)])
            if any([x is None for x in power_values]) or num_rows != 96:
                acc_energy = [None] * num_rows
                day_energy = [None] * num_rows
            else:
                acc_energy = accumulated_day_energy(power_values)
                day_energy = whole_day_energy(power_values)
                
            df.loc[(df['time'].dt.date == day) & (df['value_key'] == value_key), 'accumulated_energy_24h'] = acc_energy    
            df.loc[(df['time'].dt.date == day) & (df['value_key'] == value_key), 'whole_day_energy_24h'] = day_energy

    os.makedirs(f'../data/test_data/{site_id}', exist_ok=True)
    distinct_value_keys = df['value_key'].unique()
    for value_key in distinct_value_keys:
        filtered_df = df[df['value_key'] == value_key]
        filtered_df.to_csv(f'../data/test_data/{site_id}/{value_key}.csv', index=False)

    

    df = pd.DataFrame()
    return df, site['weather_data']

pool = Pool(8)
results = pool.map(process_site, site_info['site_id'])
df_all = pd.concat([result[0] for result in results], ignore_index=True)
site_info['weather_data'] = [result[1] for result in results]

In [ ]:
# load production data from disk

site_infos = pd.read_csv(f'../data/test_data/site_info.csv')

power_data = pd.DataFrame()

for index, row in site_infos.iterrows():
    site_id = row['site_id']
    value_keys = ['POWER_PRODUCTION']
    for value_key in value_keys:
        if os.path.exists(f'../data/test_data/{site_id}/{value_key}.csv'):
            df = pd.read_csv(f'../data/test_data/{site_id}/{value_key}.csv')
            power_data = pd.concat([power_data, df], ignore_index=True)


power_data['time'] = pd.to_datetime(power_data['time'])
df_all = power_data

site_info = site_infos

print(f'Data length: {len(power_data)}')

In [ ]:
# Get orientation clusters
# The timeframe from May to July is chosen for data collection due to the Sun's position in Central Europe. During these months, the Sun is at its highest point, 
# providing maximum sunlight exposure. This leads to more accurate and reliable data, particularly in studies related to solar energy and climate research

# get perfect day in summer months
def _get_perfect_days(df):
    # iterate over all days
    days = pd.to_datetime(df['time']).dt.date.unique()
    
    # get days in May, June, July
    days = [day for day in days if day.month in [5, 6, 7]]

    perfect_days = []

    for day in days:
        # select values between 6 am and 9 pm (6-21)
        relevant_data_of_day = df[df['time'].dt.date == day]
        if not len(relevant_data_of_day) == 96:
            # print(f'Not all values available for {day} and site {site_id}, actual length: {len(relevant_data_of_day)}')
            continue

        relevant_data_of_day = relevant_data_of_day[(relevant_data_of_day['time'].dt.hour >= 6) & (relevant_data_of_day['time'].dt.hour <= 21)]

        # get days with practically no clouds (no value over 10%) between 6 am and 9 pm (6-21)
        if not (relevant_data_of_day[relevant_data_of_day['clouds'] > 35].empty):
            # print(f'Clouds over 10% for {day} and site {site_id}, max clouds: {relevant_data_of_day["clouds"].max()}')
            continue

        perfect_days.append(day)

    print(f'Perfect days for site {site_id}: {len(perfect_days)}')       
    return perfect_days

def _median_time_of_max_values_of_perfect_days(df, column):
    perfect_days = _get_perfect_days(df)
    
    times_max_value = []
    for day in perfect_days:
        day_data = df[df_all['time'].dt.date == day]
        max_value = day_data[column].max()
        if np.isnan(max_value):
            continue
        time_max_value = day_data[day_data[column] == max_value]['time'].values[0]
        # converto to datetime object
        time_max_value = pd.to_datetime(time_max_value)
        times_max_value.append(time_max_value)

    times_max_value = [time.hour * 60 + time.minute for time in times_max_value]
    median_time = np.median(times_max_value)

    return median_time

def _median_value_for_time(df, column, hour, minute = 0):
    perfect_days = _get_perfect_days(df)
    
    values = []
    for day in perfect_days:
        day_data = df[df_all['time'].dt.date == day]
        time = pd.Timestamp(year=day.year, month=day.month, day=day.day, hour=hour, minute=minute)
        value = day_data[day_data['time'] == time][column].values
        if len(value) == 0:
            continue

        values.append(value[0])
        
    return np.median(values)

def _median_hour_max_energy_per_hour(df):
    perfect_days = _get_perfect_days(df)
    
    hour_max_value = []
    for day in perfect_days:
        day_data = df[df_all['time'].dt.date == day]
        
        max_hour = 0
        max_hour_value = 0
        for hour in range(1, 23):
            hour_energy = day_data[day_data['time'].dt.hour == hour - 1]['avg'].mean()

            if hour_energy > max_hour_value:
                max_hour = hour
                max_hour_value = hour_energy
        
        hour_max_value.append(max_hour)
        
    median_hour = np.median(hour_max_value)

    return median_hour

sites_with_max_data = []
for site_id in site_info['site_id']:
    site = site_info[site_info['site_id'] == site_id].iloc[0]
    df = df_all[(df_all['site_id'] == site_id) & (df_all['value_key'] == 'POWER_PRODUCTION')]

    site['median_time_of_max_solar_rad'] = _median_time_of_max_values_of_perfect_days(df, 'solar_rad')
    site['median_time_of_max_avg'] = _median_time_of_max_values_of_perfect_days(df, 'avg')
    site['median_hour_max_energy'] = _median_hour_max_energy_per_hour(df)

    sites_with_max_data.append(site)


In [ ]:
# order sites by median time of max values
sites_with_max_data = sorted(sites_with_max_data, key=lambda x: x['median_time_of_max_avg'])

for site in sites_with_max_data:
    print(f'Site ID: {site["site_id"]}, Median time of max solar radiation: {site["median_time_of_max_solar_rad"]}, Median time of max avg: {site["median_time_of_max_avg"]}, Median hour of max energy: {site["median_hour_max_energy"]}')


In [ ]:
print(site_info)
print()

In [ ]:
# use dbscan to cluster sites by median time of max values
sites_with_max_data = [site for site in sites_with_max_data if not np.isnan(site["median_time_of_max_avg"])]

data = np.array([site["median_time_of_max_avg"] for site in sites_with_max_data]).reshape(-1, 1)

# divide values by max (1440 minutes in a day)
data = data/1440

dbscan = DBSCAN(eps=0.01, min_samples=2)

clusters = dbscan.fit_predict(data)

for i, site in enumerate(sites_with_max_data):
    site_id = site["site_id"]
    cluster = clusters[i]
    row = site_info[site_info["site_id"] == site_id].iloc[0]
    row["clusters"]["orientation"] = cluster
    
# set -1 for sites with no orientation data
for site in site_info.iterrows():
    if "orientation" not in site[1]["clusters"]:
        site[1]["clusters"]["orientation"] = -1

print(site_info)

In [ ]:
# set actual kWp values (inferred from the data)

for site in site_info.iterrows():
    site_id = site[1]["site_id"]
    df = df_all[(df_all["site_id"] == site_id) & (df_all["value_key"] == "POWER_PRODUCTION")]
    
    max_power = df["avg"].max()

    if max_power == max_power:
        kwp = max_power / 1000
        # round up
        kwp = round(kwp + 0.5)

        site_info.loc[site[0], "kwp"] = kwp

In [ ]:
# save site_info 
site_info.to_csv(f'../data/test_data/site_info.csv', index=True)

for site in site_info.iterrows():
    site_id = site[1]["site_id"]
    site_info = site[1]
    site_info.to_csv(f'../data/test_data/{site_id}/site_info.csv', index=True)

In [7]:
window_size = 7  # The size of each window (in days)

def are_subsequent(dates):
    """Check if dates are subsequent."""
    return np.all(np.diff(dates) == pd.Timedelta(days=1))

for site_id in site_info['site_id']:
    site = site_info[site_info['site_id'] == site_id].iloc[0]
    site_power_data = power_data[(power_data['site_id'] == site_id)]

    unique_days = np.sort(site_power_data['time'].dt.date.unique())

    # Split the unique days into windows
    windows = [unique_days[i:i+window_size] for i in range(0, len(unique_days), window_size)]

    # Filter out non-subsequent windows
    complete_windows = [window for window in windows if len(window) == window_size and are_subsequent(window)]

    # Shuffle the windows
    random.shuffle(complete_windows)

    # Select a fixed number of windows for testing
    test_windows_size = 10  # Change this to the number of windows you want for testing
    test_windows, training_windows = complete_windows[:test_windows_size], complete_windows[test_windows_size:]

    # Sort the windows
    test_windows.sort(key=lambda x: x[0])
    training_windows.sort(key=lambda x: x[0])

    # Convert windows into DataFrame
    training_windows_df = pd.DataFrame(training_windows)
    test_windows_df = pd.DataFrame(test_windows)

    # Write windows to CSV files
    training_windows_df.to_csv(f'../data/test_data/{site_id}/training_windows.csv', index=False, header=True)
    test_windows_df.to_csv(f'../data/test_data/{site_id}/test_windows.csv', index=False, header=True)

In [ ]:
# print max values for solar_rad and ghi
print("Max solar_rad:", df_all['solar_rad'].max())
print("Max ghi:", df_all['ghi'].max())
print("Max temp:", df_all['temp'].max())
print("Max precip:", df_all['precip'].max())
print("Max snow depth:", df_all['snow_depth'].max())
print("Max clouds:", df_all['clouds'].max())

# min values
print("Min solar_rad:", df_all['solar_rad'].min())
print("Min ghi:", df_all['ghi'].min())